### Step 2

In [1]:
# This allows us to show the full screen width

from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

In [2]:
import pandas as pd
import numpy as np

In [3]:
BLmod = pd.read_excel("Assignment Data/Week 3/Bundesliga prediction model (Assignment).xlsx")
BLmod

,Date,gameno,day,month,year,HomeTeam,AwayTeam,FTHG,FTAG,FTR,B365H,B365D,B365A,Tmhome,Tmaway
0,16/08/2019,1,16,8,2019,Bayern Munich,Hertha,2,2,D,1.14,8.00,15.00,116.05,215.20
1,17/08/2019,2,17,8,2019,Dortmund,Augsburg,5,1,H,1.20,7.00,13.00,790.40,116.05
2,17/08/2019,3,17,8,2019,Wolfsburg,FC Koln,2,1,H,1.95,3.50,4.00,162.28,101.10
3,17/08/2019,4,17,8,2019,Werder Bremen,Fortuna Dusseldorf,1,3,A,1.75,3.75,4.75,35.58,81.65
4,17/08/2019,5,17,8,2019,Freiburg,Mainz,3,0,H,2.25,3.25,3.40,81.65,148.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,27/06/2020,302,27,6,2020,Dortmund,Hoffenheim,0,4,A,1.70,4.50,4.10,642.10,229.50
302,27/06/2020,303,27,6,2020,Leverkusen,Mainz,1,0,H,1.25,6.50,10.00,420.60,148.60
303,27/06/2020,304,27,6,2020,Ein Frankfurt,Paderborn,3,2,H,1.30,5.75,9.00,207.55,24.10
304,27/06/2020,305,27,6,2020,Augsburg,RB Leipzig,1,2,A,7.00,5.75,1.36,116.05,521.90


In [4]:
# define the predicted result based on the betting odds 

BLmod['B365res']= np.where((BLmod['B365H']<BLmod['B365D']) & (BLmod['B365H']<BLmod['B365A']),'H',\
                        np.where((BLmod['B365D']<BLmod['B365H']) & (BLmod['B365D']<BLmod['B365A']),'D',\
                        np.where((BLmod['B365A']<BLmod['B365H']) & (BLmod['B365A']<BLmod['B365D']),'A',"")))

In [5]:
# define the probabilities associated with bookmaker odds

BLmod['B365probH'] = 1/BLmod['B365H']/(1/BLmod['B365H']+1/BLmod['B365D']+1/BLmod['B365A'])
BLmod['B365probD'] = 1/BLmod['B365D']/(1/BLmod['B365H']+1/BLmod['B365D']+1/BLmod['B365A'])
BLmod['B365probA'] = 1/BLmod['B365A']/(1/BLmod['B365H']+1/BLmod['B365D']+1/BLmod['B365A'])

# we now create a winvalue (for the home team, 2 = win, 1 = draw, 0 = loss) which we will need for the order logit regression.

BLmod['winvalue'] = np.where(BLmod['FTR'] == 'H', 2 ,(np.where(BLmod['FTR'] == 'D', 1, 0)))

In [6]:
# optional crosstab showing how often the bookmaker odds correctly predict the result

pd.crosstab(BLmod['FTR'], BLmod['B365res'],dropna= True)

B365res,A,H
FTR,,
A,63,52
D,18,50
H,28,95


In [7]:
# create the log of salary ratios 
BLmod['lhTMratio'] = np.log(BLmod['Tmhome']/BLmod['Tmaway'])

In [8]:
BLtrain = BLmod[:224]
BLtrain

,Date,gameno,day,month,year,HomeTeam,AwayTeam,FTHG,FTAG,FTR,...,B365D,B365A,Tmhome,Tmaway,B365res,B365probH,B365probD,B365probA,winvalue,lhTMratio
0,16/08/2019,1,16,8,2019,Bayern Munich,Hertha,2,2,D,...,8.00,15.00,116.05,215.20,H,0.820681,0.116947,0.062372,1,-0.617547
1,17/08/2019,2,17,8,2019,Dortmund,Augsburg,5,1,H,...,7.00,13.00,790.40,116.05,H,0.791304,0.135652,0.073043,2,1.918518
2,17/08/2019,3,17,8,2019,Wolfsburg,FC Koln,2,1,H,...,3.50,4.00,162.28,101.10,H,0.489083,0.272489,0.238428,2,0.473213
3,17/08/2019,4,17,8,2019,Werder Bremen,Fortuna Dusseldorf,1,3,A,...,3.75,4.75,35.58,81.65,H,0.544933,0.254302,0.200765,0,-0.830658
4,17/08/2019,5,17,8,2019,Freiburg,Mainz,3,0,H,...,3.25,3.40,81.65,148.60,H,0.424796,0.294089,0.281115,2,-0.598816
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,2020-07-03 00:00:00,220,7,3,2020,Freiburg,Union Berlin,3,1,H,...,3.40,3.10,113.98,35.58,H,0.413495,0.279718,0.306787,2,1.164239
220,2020-07-03 00:00:00,221,7,3,2020,Hertha,Werder Bremen,2,2,D,...,3.50,3.40,215.20,162.28,H,0.450928,0.270557,0.278515,1,0.282245
221,2020-08-03 00:00:00,222,8,3,2020,Bayern Munich,Augsburg,2,0,H,...,10.00,21.00,790.40,116.05,H,0.859212,0.095373,0.045415,2,1.918518
222,2020-08-03 00:00:00,223,8,3,2020,Mainz,Fortuna Dusseldorf,1,1,D,...,3.75,3.80,148.60,81.65,H,0.498339,0.252492,0.249169,1,0.598816


In [9]:
from bevel.linear_ordinal_regression import OrderedLogit
ol = OrderedLogit()

In [10]:
ol.fit(BLtrain['lhTMratio'], BLtrain['winvalue'])
ol.print_summary()

n=224
                  beta  se(beta)      p  lower 0.95  upper 0.95     
attribute names                                                     
column_1        0.5981    0.1129 0.0000      0.3769      0.8193  ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1 

Somers' D = 0.278


In [11]:
#%% To get the coefficients and the intercepts
print(f'beta = {ol.coef_[0]:.4f}')
print(f'interceptAD = {ol.coef_[1]:.4f}')
print(f'interceptDH = {ol.coef_[2]:.4f}')

beta = 0.5981
interceptAD = -0.6734
interceptDH = 0.3356
